In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras import layers

/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Applications/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework

In [2]:
model = tf.keras.models.Sequential()
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [3]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.1),
             loss=tf.losses.categorical_crossentropy,
             metrics=[tf.metrics.categorical_accuracy])

In [4]:
train_x = np.random.random((6000, 72))
train_y = np.random.random((6000, 10))
model.fit(train_x, train_y, epochs=5)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 6000 samples
Epoch 1/5
6000/6000 [==============================] - 0s 69us/sample - loss: 11.6844 - categorical_accuracy: 0.1035
Epoch 2/5
6000/6000 [==============================] - 0s 29us/sample - loss: 11.5205 - categorical_accuracy: 0.1008
Epoch 3/5
6000/6000 [==============================] - 0s 29us/sample - loss: 11.5224 - categorical_accuracy: 0.0993
Epoch 4/5
6000/6000 [==============================] - 0s 30us/sample - loss: 11.5230 - categorical_accuracy: 0.0943
Epoch 5/5
6000/6000 [==============================] - 0s 30us/sample - loss: 11.5212 - categorical_accuracy: 0.1008


In [5]:
test_x = np.random.random((1000, 72))
test_y = np.random.random((1000, 10))
model.evaluate(test_x, test_y, batch_size=32)

1000/1000 [==============================] - 0s 53us/sample - loss: 11.4695 - categorical_accuracy: 0.0950


[11.469466491699219, 0.095]

In [6]:
test_data = tf.data.Dataset.from_tensor_slices((test_x, test_y))
test_data = test_data.batch(32).repeat()
model.evaluate(test_data, steps=30)

30/30 [==============================] - 0s 935us/step - loss: 11.4803 - categorical_accuracy: 0.0958


[11.480275122324626, 0.09583333]

In [7]:
result = model.predict(test_x, batch_size=32)
np.shape(result), result

((1000, 10),
 array([[0.10256103, 0.10695486, 0.08855996, ..., 0.10658294, 0.11195687,
         0.09660218],
        [0.10256103, 0.10695486, 0.08855996, ..., 0.10658294, 0.11195687,
         0.09660218],
        [0.10256103, 0.10695486, 0.08855996, ..., 0.10658294, 0.11195687,
         0.09660218],
        ...,
        [0.10256103, 0.10695486, 0.08855996, ..., 0.10658294, 0.11195687,
         0.09660218],
        [0.10256103, 0.10695486, 0.08855996, ..., 0.10658294, 0.11195687,
         0.09660218],
        [0.10256103, 0.10695486, 0.08855996, ..., 0.10658294, 0.11195687,
         0.09660218]], dtype=float32))

In [8]:
class MyModel(tf.keras.Model):
    def __init__(self, num_classes=10):
        super(MyModel, self).__init__(name='my_model')
        self.num_classes = num_classes
        self.layer1 = layers.Dense(32, activation='relu')
        self.layer2 = layers.Dense(num_classes, activation='softmax')
        
    def call(self, inputs):
        h1 = self.layer1(inputs)
        out = self.layer2(h1)
        return out
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.num_classes
        return tf.TensorShape(shape)

In [9]:
model = MyModel(10)
model.compile(optimizer=tf.optimizers.RMSprop(0.001),
             loss=tf.losses.categorical_crossentropy,
             metrics=['accuracy'])
model.fit(train_x, train_y, epochs=5)

Train on 6000 samples
Epoch 1/5
6000/6000 [==============================] - 0s 54us/sample - loss: 11.5546 - accuracy: 0.0983
Epoch 2/5
6000/6000 [==============================] - 0s 28us/sample - loss: 11.5138 - accuracy: 0.1105
Epoch 3/5
6000/6000 [==============================] - 0s 30us/sample - loss: 11.5132 - accuracy: 0.1053
Epoch 4/5
6000/6000 [==============================] - 0s 31us/sample - loss: 11.5128 - accuracy: 0.1055
Epoch 5/5
6000/6000 [==============================] - 0s 29us/sample - loss: 11.5126 - accuracy: 0.1063


In [10]:
class MyLayer(layers.Layer):
    def __init__(self, out_dim, **kwargs):
        self.out_dim = out_dim
        super(MyLayer, self).__init__(**kwargs)
        
    def build(self, input_shape):
        shape = tf.TensorShape((input_shape[1], self.out_dim))
        self.kernel = self.add_weight(name='kernel1', shape=shape, 
                          initializer='uniform', trainable=True)
        super(MyLayer, self).build(input_shape)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)
    
    def compute_output_shape(self, input_shape):
        shape = tf.TensorShape(input_shape).as_list()
        shape[-1] = self.out_dim
        return tf.TensorShape(shape)
    
    def get_config(self):
        base_config = super(MyLayer, self).get_config()
        base_config['output_dim'] = self.out_dim
        return base_config
    
    @classmethod
    def from_config(cls, config):
        return cls(**config)

In [11]:
model = tf.keras.Sequential([MyLayer(10), 
                             layers.Activation('softmax')])
model.compile(optimizer=tf.optimizers.RMSprop(0.001),
             loss=tf.losses.categorical_crossentropy,
             metrics=['accuracy'])
model.fit(train_x, train_y, epochs=5, batch_size=16)

Train on 6000 samples
Epoch 1/5
6000/6000 [==============================] - 0s 60us/sample - loss: 11.5272 - accuracy: 0.1055
Epoch 2/5
6000/6000 [==============================] - 0s 47us/sample - loss: 11.5199 - accuracy: 0.1015
Epoch 3/5
6000/6000 [==============================] - 0s 50us/sample - loss: 11.5166 - accuracy: 0.1087
Epoch 4/5
6000/6000 [==============================] - 0s 50us/sample - loss: 11.5147 - accuracy: 0.1108
Epoch 5/5
6000/6000 [==============================] - 0s 51us/sample - loss: 11.5132 - accuracy: 0.1180


In [12]:
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2, monitor='loss'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(train_x, train_y, batch_size=16, epochs=5,
         callbacks=callbacks)

Train on 6000 samples
Epoch 1/5
6000/6000 [==============================] - 0s 51us/sample - loss: 11.5128 - accuracy: 0.1202
Epoch 2/5
6000/6000 [==============================] - 0s 50us/sample - loss: 11.5129 - accuracy: 0.1265
Epoch 3/5
6000/6000 [==============================] - 0s 52us/sample - loss: 11.5128 - accuracy: 0.1168


In [17]:
model = tf.keras.models.Sequential([
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
model.compile(optimizer=tf.optimizers.Adam(0.001),
             loss=tf.losses.categorical_crossentropy,
             metrics=['accuracy'])
model.fit(train_x, train_y, epochs=5)
model.save_weights('./weights/model')
model.load_weights('./weights/model')
model.save_weights('./weights/weight.h5')
model.load_weights('./weights/weight.h5')

Train on 6000 samples
Epoch 1/5
6000/6000 [==============================] - 0s 52us/sample - loss: 11.6139 - accuracy: 0.0993
Epoch 2/5
6000/6000 [==============================] - 0s 31us/sample - loss: 11.5355 - accuracy: 0.1003
Epoch 3/5
6000/6000 [==============================] - 0s 30us/sample - loss: 11.5247 - accuracy: 0.1002
Epoch 4/5
6000/6000 [==============================] - 0s 31us/sample - loss: 11.5179 - accuracy: 0.1057
Epoch 5/5
6000/6000 [==============================] - 0s 30us/sample - loss: 11.5149 - accuracy: 0.1092


In [14]:
import json
import pprint
json_str = model.to_json()
# pprint.pprint(json.loads(json_str))
fresh_model = tf.keras.models.model_from_json(json_str)

yaml_str = model.to_yaml()
# print(yaml_str)
fresh_model = tf.keras.models.model_from_yaml(yaml_str)

In [20]:
model = tf.keras.Sequential([
  layers.Dense(10, activation='softmax', input_shape=(72,)),
  layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_x, train_y, batch_size=32, epochs=5)

import os
os.mkdir('models')
model.save('./models/model.h5')
fresh_model = tf.keras.models.load_model('./models/model.h5')

Train on 6000 samples
Epoch 1/5
6000/6000 [==============================] - 0s 54us/sample - loss: 11.5194 - accuracy: 0.1013
Epoch 2/5
6000/6000 [==============================] - 0s 29us/sample - loss: 11.5164 - accuracy: 0.1067
Epoch 3/5
6000/6000 [==============================] - 0s 29us/sample - loss: 11.5153 - accuracy: 0.1057
Epoch 4/5
6000/6000 [==============================] - 0s 28us/sample - loss: 11.5147 - accuracy: 0.1072
Epoch 5/5
6000/6000 [==============================] - 0s 29us/sample - loss: 11.5140 - accuracy: 0.1130


In [16]:
estimator = tf.keras.estimator.model_to_estimator(model)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using the Keras model provided.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/zd/dj641pks6fsdt5lj4zjp0m4w0000gn/T/tmpy4tegvuw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0xb286b6fd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
